In [ ]:
from pyspark.sql import SparkSession,functions as F
from pyspark import StorageLevel
from pyspark import SparkConf
from pyspark import SparkConf

conf = SparkConf()

conf.set("spark.app.name", "ComprehensiveSparkJob") 
conf.set("spark.master", "local[*]")                
conf.set("spark.driver.memory", "4g")      
conf.set("spark.driver.cores", "1")                 
conf.set("spark.ui.port", "4040")                

# Executor Settings
conf.set("spark.executor.memory", "2g")      
conf.set("spark.executor.cores", "2")               
conf.set("spark.executor.instances", "3")         


conf.set("spark.default.parallelism", "6")         
conf.set("spark.sql.shuffle.partitions", "6")       
conf.set("spark.task.cpus", "1")  

# Data Handling Settings
conf.set("spark.memory.fraction", "0.8")            
conf.set("spark.memory.storageFraction", "0.5")     

# Serialization Settings
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")  
conf.set("spark.kryo.registrationRequired", "true")  
conf.set("spark.kryo.classesToRegister", "org.apache.spark.sql.Row")  

# Debugging and Logging
conf.set("spark.eventLog.enabled", "true")           
conf.set("spark.eventLog.dir", "/root/spark_log/spark-events/")  
conf.set("spark.history.fs.logDirectory", "/root/spark_log/spark-history/")  


In [ ]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [3]:
input_json_file_path = "/root/docker_dataset/json_splits/"  
input_file_list  = rfl(input_json_file_path,".json")
parquet_file_path = "/root/docker_dataset/json_splits/table_example_1.parquet"


In [4]:

def func_cache_dft_data(dataframe) :
    dataframe.persist(StorageLevel.MEMORY_AND_DISK)

def func_json_dataframe (input_file:str) :
    return  spark.read.option("multiline", "true").json(input_file)

def func_repartion_dataframe(dataframe, repartion_count) :
    return dataframe.repartition(repartion_count)
 

def func_explode_based_on_one_key (input_dataframe,explode_key:str , alias_of_column :str)  :
    return  input_dataframe.select(F.explode(F.col(explode_key)).alias(alias_of_column))

def func_explode_based_on_depper_root_key (input_dataframe,explode_key:list  , drop_column :str)  :
    """ Always try to assemeble  he json key  based on the root level  in heirarchial order like root -->values-->payload-->commits then the explode_key   value should be [root,values,payload,commits]"""
    column_level_analysis = (".").join(explode_key)
    return  input_dataframe.select(f"{explode_key[0]}.*",F.explode_outer(F.col(column_level_analysis)).alias(column_level_analysis.replace(".","_"))).drop(drop_column)


def func_get_rdd_num_partitions( dataframe) :
    return dataframe.rdd.getNumPartitions()

def func_write_data_to_parquet ( file_path, dataframe,write_mode) :
    dataframe.write.parquet(file_path,mode=write_mode)


def func_read_parquet_file(file_path) :
    return spark.read.parquet(file_path)

def func_stop_spark () :
      spark.stop()


def func_remane_column_names (dataframe) :
    column_list = [F.col(c).alias(c.replace('.', '_')) for c in dataframe.columns]
    for   values in column_list:
        print(values)
    return  dataframe.select([F.col(c).alias(c.replace('.', '_')) for c in dataframe.columns])


def  func_limit_rows_dataframe (dataframe, limit_row_count) :
    return dataframe. limit(limit_row_count)

def  finc_count_dft_rows(dataframe):
     return dataframe.count()



In [5]:

def func_main () :
     processed_dft_row_count= 0 
     for file_list in input_file_list :
         big_dpart_dft = func_json_dataframe (input_json_file_path+file_list)
         big_dpart_dft = func_repartion_dataframe(big_dpart_dft,100)
         big_dpart_dft_json_normalised = func_explode_based_on_one_key (big_dpart_dft,"values","table_example")
         big_dpart_dft_json_normalised = func_explode_based_on_depper_root_key(big_dpart_dft_json_normalised,["table_example","entities"],"entities")
         func_cache_dft_data(big_dpart_dft_json_normalised)
         print(func_get_rdd_num_partitions(big_dpart_dft_json_normalised))
         big_dpart_dft_json_norm_renamed  = func_remane_column_names(big_dpart_dft_json_normalised)
         processed_dft_row_count += finc_count_dft_rows(big_dpart_dft_json_norm_renamed)
         func_write_data_to_parquet(parquet_file_path,big_dpart_dft_json_norm_renamed,"append")
     return processed_dft_row_count

In [ ]:
try :
    # json_input_row_count  = func_main () 
    big_dpart_parquet = func_read_parquet_file(parquet_file_path)
    # parquet_row_count  = finc_count_dft_rows(big_dpart_parquet)
    # big_dpart_parquet = func_limit_rows_dataframe(big_dpart_parquet,100)
    big_dpart_parquet.show()
except  Exception as error :
    spark.stop()
    raise error 

In [ ]:
db_url = "jdbc:postgresql://host.docker.internal:5432/crime_data_la"
db_properties = {
    "user": "john_user",
    "password": "abc@12345"
}

table_name = "crime_data.transactions_data"
df_table = spark.read.jdbc(url=db_url, table=table_name, properties=db_properties)
df_table.show()


In [ ]:
df_table.describe

In [ ]:
big_dpart_parquet.select(big_dpart_parquet["table_example_entities"])

In [3]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Read CSV with PySpark") \
    .getOrCreate()

# Define the file path
file_path = '/root/docker_dataset/h1b_data/h1b17.csv'

# Read the CSV file into a DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)

# List of columns to exclude
exclude_columns = ['column_to_exclude1', 'column_to_exclude2']

# Select only the columns you want to keep
selected_columns = [col for col in df.columns if col not in exclude_columns]

print(selected_columns)
df_selected = df.select(*selected_columns)

# Show the resulting DataFrame
df_selected.show()

['_c0', 'CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE', 'VISA_CLASS', 'EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE', 'EMPLOYER_NAME', 'EMPLOYER_BUSINESS_DBA', 'EMPLOYER_ADDRESS', 'EMPLOYER_CITY', 'EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE', 'EMPLOYER_COUNTRY', 'EMPLOYER_PROVINCE', 'EMPLOYER_PHONE', 'EMPLOYER_PHONE_EXT', 'AGENT_REPRESENTING_EMPLOYER', 'AGENT_ATTORNEY_NAME', 'AGENT_ATTORNEY_CITY', 'AGENT_ATTORNEY_STATE', 'JOB_TITLE', 'SOC_CODE', 'SOC_NAME', 'NAICS_CODE', 'TOTAL_WORKERS', 'NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT', 'CHANGE_PREVIOUS_EMPLOYMENT', 'NEW_CONCURRENT_EMPLOYMENT', 'CHANGE_EMPLOYER', 'AMENDED_PETITION', 'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY', 'PW_WAGE_LEVEL', 'PW_SOURCE', 'PW_SOURCE_YEAR', 'PW_SOURCE_OTHER', 'WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY', 'H1B_DEPENDENT', 'WILLFUL_VIOLATOR', 'SUPPORT_H1B', 'LABOR_CON_AGREE', 'PUBLIC_DISCLOSURE_LOCATION', 'WORKSITE_CITY', 'WORKSITE_COUNTY', 'WORKSITE_STATE', 'WORK

25/02/07 20:25:30 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/02/07 20:25:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , CASE_NUMBER, CASE_STATUS, CASE_SUBMITTED, DECISION_DATE, VISA_CLASS, EMPLOYMENT_START_DATE, EMPLOYMENT_END_DATE, EMPLOYER_NAME, EMPLOYER_BUSINESS_DBA, EMPLOYER_ADDRESS, EMPLOYER_CITY, EMPLOYER_STATE, EMPLOYER_POSTAL_CODE, EMPLOYER_COUNTRY, EMPLOYER_PROVINCE, EMPLOYER_PHONE, EMPLOYER_PHONE_EXT, AGENT_REPRESENTING_EMPLOYER, AGENT_ATTORNEY_NAME, AGENT_ATTORNEY_CITY, AGENT_ATTORNEY_STATE, JOB_TITLE, SOC_CODE, SOC_NAME, NAICS_CODE, TOTAL_WORKERS, NEW_EMPLOYMENT, CONTINUED_EMPLOYMENT, CHANGE_PREVIOUS_EMPLOYMENT, NEW_CONCURRENT_EMPLOYMENT, CHANGE_EMPLOYER, AMENDED_PETITION, FULL_TIME_POSITION, PREVAILING_WAGE, PW_UNIT_OF_PAY, PW_WAGE_LEVEL, PW_SOURCE, PW_SOURCE_YEAR, PW_SOURCE_OTHER, WAGE_RATE_OF_PAY_FROM, WAGE_RATE_OF_PAY

+---+------------------+-------------------+--------------+-------------+----------+---------------------+-------------------+--------------------+---------------------+--------------------+-------------+--------------+--------------------+--------------------+-----------------+--------------+------------------+---------------------------+-------------------+-------------------+--------------------+--------------------+--------+--------------------+----------+-------------+--------------+--------------------+--------------------------+-------------------------+---------------+----------------+------------------+---------------+--------------+-------------+---------+--------------+--------------------+---------------------+-------------------+----------------+-------------+----------------+-----------+---------------+--------------------------+-------------+---------------+--------------+--------------------+------------------+
|_c0|       CASE_NUMBER|        CASE_STATUS|CASE_SUBMITTED|